In [1]:
import polars as pl
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cart_adds = pl.read_parquet("/content/drive/MyDrive/recsys/bert/full_data/page_visit.parquet")

In [6]:
cart_adds

client_id,timestamp,url
i64,str,i64
13906876,"""2022-07-13 10:37:45""",12160626
13906876,"""2022-08-11 18:13:20""",9464384
7825390,"""2022-05-26 10:28:05""",3876331
13416433,"""2022-05-25 18:20:10""",15017297
13416433,"""2022-06-03 20:03:20""",13003066
…,…,…
12296853,"""2022-10-07 08:07:30""",13289842
12296853,"""2022-10-07 08:07:40""",13289842
20706831,"""2022-07-30 13:18:25""",14918549


In [7]:
from collections import defaultdict


def create_inputs(target):
    """inputs creation

    Args:
        target (pl.DataFrame): train_target dataframe
    """
    target.sort("timestamp")
    inputs = defaultdict(list)
    for row in target.iter_rows(named=True):
        client_id = row["client_id"]
        sku = row["url"]
        inputs[client_id].append(sku)

    # for client in inputs:
    #     inputs[client].append(-1)



    clients, inputs = zip(*inputs.items())
    return clients, inputs

In [8]:
clients, inputs = create_inputs(cart_adds)

import pickle

with open('/content/drive/MyDrive/recsys/bert/full_data/pagevisit_bert_train.pkl', 'wb') as f:
    pickle.dump(inputs, f)

with open('/content/drive/MyDrive/recsys/bert/full_data/pagevisit_clients_train.pkl', 'wb') as f:
    pickle.dump(clients, f)

FileNotFoundError: [Errno 2] No such file or directory: 'content/drive/MyDrive/recsys/bert/full_data/pagevisit_bert_train.pkl'

In [9]:
with open('/content/drive/MyDrive/recsys/bert/full_data/pagevisit_bert_train.pkl', 'wb') as f:
    pickle.dump(inputs, f)

with open('/content/drive/MyDrive/recsys/bert/full_data/pagevisit_clients_train.pkl', 'wb') as f:
    pickle.dump(clients, f)

In [ ]:
# %%
import polars as pl
import pickle
import os

os.makedirs('full_data/bert', exist_ok=True)

all_clients = []
all_inputs = []

# Process in client_id prefix chunks
for digit in range(10):
    print(f"Processing clients starting with '{digit}'...")

    df_chunk = (
        pl.read_parquet("full_data/page_visit.parquet", low_memory=True)
        .filter(pl.col("client_id").cast(pl.Utf8).str.starts_with(str(digit)))
        .sort("timestamp")
        .group_by("client_id")
        .agg([pl.col("url").alias("urls")])
    )

    clients_chunk = df_chunk["client_id"].to_list()
    inputs_chunk = df_chunk["urls"].to_list()

    all_clients.extend(clients_chunk)
    all_inputs.extend(inputs_chunk)

    # Optionally save individual chunks
    with open(f'full_data/bert/pagevisit_bert_train_{digit}.pkl', 'wb') as f:
        pickle.dump(inputs_chunk, f)

    with open(f'full_data/bert/pagevisit_clients_train_{digit}.pkl', 'wb') as f:
        pickle.dump(clients_chunk, f)

# Save the merged full dataset
with open('full_data/bert/pagevisit_bert_train.pkl', 'wb') as f:
    pickle.dump(all_inputs, f)

with open('full_data/bert/pagevisit_clients_train.pkl', 'wb') as f:
    pickle.dump(all_clients, f)
